In [23]:
import numpy as np

np.set_printoptions(linewidth=200)


data = """
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################"""

maze = np.array([list(r) for r in data.strip().split("\n")])

maze

array([['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
       ['#', '.', '.', '.', '#', '.', '.', '.', '#', '.', '.', '.', '#', '.', '.', 'E', '#'],
       ['#', '.', '#', '.', '#', '.', '#', '.', '#', '.', '#', '.', '#', '.', '#', '.', '#'],
       ['#', '.', '#', '.', '#', '.', '#', '.', '.', '.', '#', '.', '.', '.', '#', '.', '#'],
       ['#', '.', '#', '.', '#', '.', '#', '.', '#', '#', '#', '.', '#', '.', '#', '.', '#'],
       ['#', '.', '.', '.', '#', '.', '#', '.', '#', '.', '.', '.', '.', '.', '#', '.', '#'],
       ['#', '.', '#', '.', '#', '.', '#', '.', '#', '.', '#', '#', '#', '#', '#', '.', '#'],
       ['#', '.', '#', '.', '.', '.', '#', '.', '#', '.', '#', '.', '.', '.', '.', '.', '#'],
       ['#', '.', '#', '.', '#', '#', '#', '#', '#', '.', '#', '.', '#', '#', '#', '.', '#'],
       ['#', '.', '#', '.', '#', '.', '.', '.', '.', '.', '.', '.', '#', '.', '.', '.', '#'],
       ['#', '.', '#', '.', '#', '#', '#', '.', '#', '#', '#

In [24]:
from collections import deque, defaultdict


def shortest_path(maze): 
    sr, sc = tuple(zip(*np.where(maze == "S")))[0]
    er, ec = tuple(zip(*np.where(maze == "E")))[0]
    
    unseen = set()
    starts = set()
    arrive = {(-1, 0): '^', (0, -1): '<', (1, 0): 'v', (0, 1): '>'} # if . at k, v is possible
    for ur, uc in list(zip(*np.where(np.isin(maze, (".", "S", "E"))))):
        for ar, ac in arrive.keys():
            if maze[ur + ar, uc + ac] in (".", "S", "E"):
                ud = arrive[(ar, ac)]
                if (ur, uc) == (sr, sc):
                    starts.add((ur, uc, ud))
                    unseen.add((ur, uc, ud))
                elif (ur, uc) == (er, ec):
                    end = (ur, uc, None)
                    unseen.add((ur, uc, None))
                else:
                    unseen.add((ur, uc, ud))

    leave = {v: k for k, v in arrive.items()}
    graph = defaultdict(list)
    for ur, uc, ud in unseen - {end}:
        ar, ac = leave[ud]
        if (ur + ar, uc + ac, None) == end:
                graph[(ur, uc, ud)].append(end)
        for ad in leave.keys():
            if (ur + ar, uc + ac, ad) in unseen and ud + ad not in ("^v", "v^", "<>", "><"):
                graph[(ur, uc, ud)].append((ur + ar, uc + ac, ad))
    
    from_start = {(cr, cc, cd): (0 if (cr, cc, cd) in starts else np.inf) for cr, cc, cd in unseen}
    prev = {(cr, cc, cd): list() for (cr, cc, cd) in unseen}

    while unseen:
        unseen_dict = {u: from_start[u] for u in unseen}
        (cr, cc, cd) = min(unseen_dict, key=unseen_dict.get)
        unseen.remove((cr, cc, cd))

        if from_start[(cr, cc, cd)] == np.inf:
            break

        adjacent = graph[(cr, cc, cd)]
 
        for nr, nc, nd in adjacent:
            dist = 1 + 1000 * (1 if cd + str(nd) in ("^>", ">^", "^<", "<^", "v>", ">v", "v<", "<v") else 0)
            if from_start[(cr, cc, cd)] + dist <= from_start[(nr, nc, nd)]:
                from_start[(nr, nc, nd)] = from_start[(cr, cc, cd)] + dist
                prev[(nr, nc, nd)].append((cr, cc, cd))

        if (cr, cc, cd) == end:
            break

    stack = [(end, [end])]
    paths = list()
    while stack:
        (cr, cc, cd), path = stack.pop()
        if (cr, cc, cd) in starts and path not in paths:
            paths.append(path)
        else:
            for nr, nc, nd in prev[(cr, cc, cd)]:
                if (nr, nc, nd) not in path:
                    stack.append(((nr, nc, nd), path + [(nr, nc, nd)]))

    path = paths[0][::-1]
    fd = ">"
    turns = 0
    steps = 0
    for _, _, pd in path[:-1]:
        if pd != fd:
            turns += 1
            fd = pd
        steps += 1
    
    return turns * 1000 + steps, paths, len(set([p[:2] for path in paths for p in path]))


score, paths, count = shortest_path(maze)

score, count

(11048, 64)

In [12]:
# from collections import defaultdict


# def score_path(path):
#     sr, sc = path[0]
#     path = [(sr, sc - 1)] + path
#     dist = 0
#     for (pr, pc), (cr, cc), (nr, nc) in zip(path[:-2], path[1:-1], path[2:]):
#         vec1 = (cr - pr, cc - pc)
#         vec2 = (nr - cr, nc - cc)
#         dist += 1 + 1000 * (1 if np.dot(vec1, vec2) == 0 else 0)
#     return dist
        


# def dfs_maze(maze):
#     sr, sc = list(zip(*np.where(maze == "S")))[0]
#     er, ec = list(zip(*np.where(maze == "E")))[0]

#     stack = [((sr, sc), [(sr, sc)])]
#     paths = list()

#     while stack:
#         (cr, cc), path = stack.pop()

#         if (cr, cc) == (er, ec) and path not in paths:
#             paths.append(path)
#         else:
#             around = ((-1, 0), (0, -1), (1, 0), (0, 1))
#             adjacent = [
#                 (cr + ar, cc + ac) for ar, ac in around 
#                 if maze[cr + ar, cc + ac] in (".", "E")
#             ]
#             for nr, nc in adjacent:
#                 if (nr, nc) not in path and score_path(path) <= score:
#                     stack.append(((nr, nc), path + [(nr, nc)]))

#     return len(set([p for path in paths for p in path]))


# dfs_maze(maze)